## Checkerboard Calibration, Mean Real World error across all and across bins

In [5]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [6]:
dirs = ["./180/"]

In [4]:
if not (os.path.exists("./vid_hard_check/")):
    os.makedirs("./vid_hard_check/")

In [12]:
global_dict_no_eq = {}
for d in dirs:
    flist = sorted(glob.glob(os.path.join(d, "*.mp4")))
    for vf in flist:
        print(vf)
        vidcap = cv2.VideoCapture(vf)

        files = glob.glob("./vid_hard_check/*")
        for fl in files:
            os.remove(fl)

        success,image = vidcap.read()
        count = 0
        success = True
        while success:
            success,image = vidcap.read()
            cv2.imwrite("./vid_hard_check/frame%d.png" % count, image)     # save frame as PNG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
            count += 1

        print(count)
        count = 512

        nx = 5
        ny = 5
        bins = 6
        

        interval = count/bins

        mesh = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
        objp = np.hstack((mesh, np.zeros(shape = (nx*ny,1))))
        objp = np.array(objp, dtype = np.float32)*20.0

        fname = glob.glob('./vid_hard_check/*.png')
        
        for j in range(1,4):

            imgpoints = []
            objpoints = []
            imgpoints2_list = []
            detected_list = []
            bin_error = []
            bincount = np.zeros(6)
            fx = j*0.1
            fy = j*0.1
            for i in range(len(fname)):
                try:
                    f = "./vid_hard_check/frame{}.png".format(i+1)
                    imcv = cv2.imread(f)
                    imcv = cv2.cvtColor(imcv, cv2.COLOR_BGR2GRAY)
                    
####### Uncomment below two lines to apply histogram equalization for removing shiny effects, cl = 2, 5 or 10 work best #####                    

#                     cli = cv2.createCLAHE(clipLimit = cl)
#                     imcv = cli.apply(imcv)

                    imcv = cv2.GaussianBlur(imcv, ksize = (5,5), sigmaX = 1, sigmaY = 1)
                    imcv = cv2.resize(imcv, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)
                    img = imcv

                    retval, corners = cv2.findChessboardCorners(imcv,(nx,ny), None)
                    if (retval == True):

                        detected_list.append(f)            
###### Uncomment below lines to plot the detected checkerboard corners #######

#                         plt.figure()
#                         cv2.drawChessboardCorners(imcv, (nx, ny), corners, retval)
#                         plt.imshow(imcv)
                        imgpoints.append(corners)
                        objpoints.append(objp)

                        bincount[int(i/interval)] += 1
                              
                except Exception as e:
                    print("Exception occured {}".format(e))

            print(fx, len(objpoints))
            mean_error = 1.0
            mtx = dist = rvecs = tvecs = []
            if (len(objpoints) != 0):
                retval, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1], None, None)
                mean_error = 0.0
                
                tot_len = len(objpoints)
                st = 0
                for b in range(bins):
                    berror = 0.0
                    if b != 0 :
                        st += bincount[b-1]
                    else: 
                        st = 0
                    end = st + bincount[b]
                    for i in range(int(st),int(end)):
                        sp = imgpoints[i]
                        objp = objpoints[i]
                        src = np.array([[[sp[0,0,0], sp[0,0,1]], [sp[4,0,0], sp[4,0,1]], [sp[24,0,0], sp[24,0,1]], [sp[20,0,0], sp[20,0,1]]]],dtype=np.float32)
                        dst = np.array([[[objp[0,0], objp[0,1]], [objp[4,0], objp[4,1]], [objp[24,0], objp[24,1]], [objp[20,0], objp[20,1]]]],dtype=np.float32)
                        PM = cv2.getPerspectiveTransform(src,dst)
                        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                        sp = np.reshape(sp, newshape=(1,sp.shape[0], sp.shape[2]))
                        dp = np.reshape(imgpoints2, newshape=(1,imgpoints2.shape[0], 2))
                        slanded_real = cv2.perspectiveTransform(sp,PM)
                        dlanded_real = cv2.perspectiveTransform(dp,PM)
                        res = cv2.norm(slanded_real, dlanded_real, cv2.NORM_L2)
                        res = np.sqrt(res**2/slanded_real.shape[1])
                        berror += res
                    if bincount[b] != 0:
                        bin_error.append(berror/bincount[b])
                    else:
                        bin_error = 1.0

                for i in range(len(objpoints)):
                    sp = imgpoints[i]
                    objp = objpoints[i]
                    src = np.array([[[sp[0,0,0], sp[0,0,1]], [sp[4,0,0], sp[4,0,1]], [sp[24,0,0], sp[24,0,1]], [sp[20,0,0], sp[20,0,1]]]],dtype=np.float32)
                    dst = np.array([[[objp[0,0], objp[0,1]], [objp[4,0], objp[4,1]], [objp[24,0], objp[24,1]], [objp[20,0], objp[20,1]]]],dtype=np.float32)
                    PM = cv2.getPerspectiveTransform(src,dst)
                    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                    sp = np.reshape(sp, newshape=(1,sp.shape[0], sp.shape[2]))
                    dp = np.reshape(imgpoints2, newshape=(1,imgpoints2.shape[0], 2))
                    slanded_real = cv2.perspectiveTransform(sp,PM)
                    dlanded_real = cv2.perspectiveTransform(dp,PM)
                    res = cv2.norm(slanded_real, dlanded_real, cv2.NORM_L2)
                    res = np.sqrt(res**2/slanded_real.shape[1])
                    mean_error += res
                print( "total error: {} for scaling of {} with {} number of images for file {}".format(mean_error/len(objpoints), fx, len(objpoints), vf))

            if vf not in global_dict_no_eq:
                global_dict_no_eq[vf] = []

            global_dict_no_eq[vf].append([fx, len(objpoints), mean_error/(len(objpoints)+0.00001), mtx, dist,detected_list,imgpoints, bin_error, bincount])

./180/c1_180.mp4
512
0.1 161


AttributeError: 'float' object has no attribute 'append'

In [29]:
pickle_out = open("180_4_real.pickle","wb")
pickle.dump(global_dict_no_eq, pickle_out)
pickle_out.close()

## Plotting Real world error across bins

In [30]:
plot_dict1 = {}
plot_dict2 = {}
plot_dict3 = {}


bin_dict1 = {}
bin_dict2 = {}
bin_dict3 = {}


In [31]:
for key in global_dict_no_eq:
    l = global_dict_no_eq[key]
    plot_dict1[key] = l[0][7]
    bin_dict1[key] = l[0][8]
    plot_dict2[key] = l[1][7]
    bin_dict2[key] = l[1][8]
    plot_dict3[key] = l[2][7]
    bin_dict3[key] = l[2][8]

In [33]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets

py.offline.init_notebook_mode(connected=True)

In [34]:
layout1 = go.Layout(
    title='Real RMSE Error Plot(fx = 0.1)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace11 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace12 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace13 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace14 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout2 = go.Layout(
    title='Real RMSE Error Plot(fx = 0.2)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace21 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace22 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace23 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace24 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout3 = go.Layout(
    title='Real RMSE Error Plot(fx = 0.3)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace31 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace32 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace33 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace34 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c4_180.mp4'],
    mode='lines+markers'
)

In [35]:
fig1 = go.Figure(data=[trace11,trace12,trace13,trace14], layout=layout1)
fig2 = go.Figure(data=[trace21,trace22,trace23,trace24], layout=layout2)
fig3 = go.Figure(data=[trace31,trace32,trace33,trace34], layout=layout3)

In [36]:
py.offline.iplot(fig1)
py.offline.iplot(fig2)
py.offline.iplot(fig3)

In [37]:
layout1 = go.Layout(
    title='Images detected Plot(fx = 0.1)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace11 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace12 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace13 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace14 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout2 = go.Layout(
    title='Images detected Plot(fx = 0.2)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace21 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace22 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace23 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace24 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout3 = go.Layout(
    title='Images detected Plot(fx = 0.3)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace31 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace32 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace33 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace34 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c4_180.mp4'],
    mode='lines+markers'
)

fig1 = go.Figure(data=[trace11,trace12,trace13,trace14], layout=layout1)
fig2 = go.Figure(data=[trace21,trace22,trace23,trace24], layout=layout2)
fig3 = go.Figure(data=[trace31,trace32,trace33,trace34], layout=layout3)

py.offline.iplot(fig1)
py.offline.iplot(fig2)
py.offline.iplot(fig3)